In [2]:
import pandas as pd
import numpy as np
import os

data = pd.read_csv(r'C:\Users\majidmm\Documents\FDD\combined_dsp_station_valid_count\DSPFleetAudit142.csv', encoding='iso-8859-1')

data['Document Quality'] = data['Document Quality'].astype('str')
data['Duplicate'] = data['Duplicate'].astype('str')

data['ownershiptype'] = data['ownershiptype'].astype('str')

data['identifier'] = data['identifier'].astype('str')

data['Rental Out Date '] = pd.to_datetime(data['Rental Out Date '],errors='coerce')
data['Rental Due Date'] = pd.to_datetime(data['Rental Due Date'], errors='coerce')

data = data[data['Document Quality']=='Valid']

data = data[~((data['Duplicate']=='Yes') | (data['Duplicate']=='yes'))]

dateRange = pd.date_range(start='07/01/2020', end='08/15/2020')

for i in dateRange:
    data.loc[(data['Rental Out Date '] <= i) & (data['Rental Due Date'] >= i), str(i)] = True
    
listFrames = []

for i in dateRange:
    temp = data[data[str(i)]==True]
    vals = temp.groupby('DSP,DS pair', as_index=False).agg({'identifier':'nunique'})
    vals.rename(columns={'identifier':str(i)}, inplace=True)
    listFrames.append(vals)
    
from functools import reduce

data = reduce(lambda left,right: pd.merge(left,right,on=['DSP,DS pair'],
                                            how='outer'), listFrames).fillna(0)

data.to_csv(r'C:\Users\majidmm\Documents\FDD\combined_dsp_station_valid_count\valid_receipts_data.xlsx', index=False)

data1=(data.set_index(['DSP,DS pair'])
   .rename_axis(['Date'], axis=1)
   .stack()
   .reset_index())
data1.rename(columns={0:'Valid Receipt Count'}, inplace=True)
data1['Valid Receipt Count']=data1['Valid Receipt Count'].astype(int)
data1['Date'] = pd.to_datetime(data1['Date'])  

data1['DSP CODE'], data1['STATION CODE'] = data1['DSP,DS pair'].str.split(',', 1).str
data1['Week'] = data1['Date'].dt.week
data1['Date'] = data1['Date'].dt.date


data1.to_excel(r'C:\Users\majidmm\Documents\FDD\combined_dsp_station_valid_count\valid_receipts_data.xlsx',index=False,columns=('DSP CODE','Week','STATION CODE','Date','Valid Receipt Count'))


data1


C:\Users\majidmm\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


,"DSP,DS pair",Date,Valid Receipt Count,DSP CODE,STATION CODE,Week
0,"18CR,DPP1",2020-07-01,8,18CR,DPP1,27
1,"18CR,DPP1",2020-07-02,8,18CR,DPP1,27
2,"18CR,DPP1",2020-07-03,8,18CR,DPP1,27
3,"18CR,DPP1",2020-07-04,8,18CR,DPP1,27
4,"18CR,DPP1",2020-07-05,8,18CR,DPP1,27
...,...,...,...,...,...,...
66235,"SUNX,DOK4",2020-08-11,0,SUNX,DOK4,33
66236,"SUNX,DOK4",2020-08-12,0,SUNX,DOK4,33
66237,"SUNX,DOK4",2020-08-13,0,SUNX,DOK4,33
66238,"SUNX,DOK4",2020-08-14,0,SUNX,DOK4,33
